In [10]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from prepare import Prepare
from split_get_scale import SplitGetScale
from scipy import stats
from scipy.stats import pearsonr
pd.options.display.max_columns = 30

In [11]:
zillow = Prepare().wrangle_zillow()

In [12]:
sgs = SplitGetScale()
train, validate, test = sgs.split(zillow)

In [13]:
train.head(3)

,parcel_id,bathroom_count,bedroom_count,home_size_square_feet,fips,latitude,longitude,lot_size_square_feet,raw_census_tract_and_block,region_id_city,region_id_county,region_id_zip,year_built,structure_tax_value_price,tax_value_dollar_count,assessment_year,land_tax_value_price,tax_amount,census_tract_and_block,log_error,transaction_date,county
13527,11770180,1.0,2,941.0,6037,34012117.0,-118247150.0,6338.0,6.037228e+07,12447,3101,95992,1909,32430.0,72079.0,2016,39649.0,1025.09,6.037228e+13,-0.091213,2017-03-01,Los Angeles
52447,10884378,3.0,3,1560.0,6037,34203300.0,-118430000.0,7313.0,6.037127e+07,12447,3101,96414,2007,166541.0,363890.0,2016,197349.0,4473.98,6.037127e+13,-0.031320,2017-06-30,Los Angeles
2559,11749839,1.0,5,2986.0,6037,34048146.0,-118311705.0,12490.0,6.037213e+07,12447,3101,96000,1921,43056.0,78654.0,2016,35598.0,1193.71,6.037213e+13,0.059301,2017-01-12,Los Angeles


In [14]:
validate.head(3)

,parcel_id,bathroom_count,bedroom_count,home_size_square_feet,fips,latitude,longitude,lot_size_square_feet,raw_census_tract_and_block,region_id_city,region_id_county,region_id_zip,year_built,structure_tax_value_price,tax_value_dollar_count,assessment_year,land_tax_value_price,tax_amount,census_tract_and_block,log_error,transaction_date,county
55329,14416991,2.0,3,1446.0,6059,33532987.0,-117723144.0,5940.0,6.059042e+07,25459,1286,96987,1968,78578.0,585076.0,2016,506498.0,5939.26,6.059042e+13,0.000431,2017-07-08,Orange
59507,12443139,2.0,3,1541.0,6037,33883181.0,-118138237.0,8795.0,6.037554e+07,44116,3101,96208,1949,40982.0,66637.0,2016,25655.0,1078.38,6.037554e+13,0.012300,2017-07-21,Los Angeles
7968,14157323,2.5,3,2085.0,6059,33925552.0,-117844599.0,3178.0,6.059022e+07,17150,1286,97037,2001,232515.0,655000.0,2016,422485.0,8323.90,6.059022e+13,0.037556,2017-02-05,Orange


In [15]:
test.head(3)

,parcel_id,bathroom_count,bedroom_count,home_size_square_feet,fips,latitude,longitude,lot_size_square_feet,raw_census_tract_and_block,region_id_city,region_id_county,region_id_zip,year_built,structure_tax_value_price,tax_value_dollar_count,assessment_year,land_tax_value_price,tax_amount,census_tract_and_block,log_error,transaction_date,county
53988,12890932,2.0,3,1460.0,6037,34068177.0,-117766814.0,10814.0,6.037402e+07,20008,3101,96508,1952,51230.0,69716.0,2016,18486.0,1059.44,6.037402e+13,-0.033498,2017-07-05,Los Angeles
46376,10859410,2.0,2,1309.0,6037,34144673.0,-118432779.0,6435.0,6.037141e+07,12447,3101,96424,1950,177425.0,526834.0,2016,349409.0,6399.10,6.037141e+13,-0.171795,2017-06-14,Los Angeles
29651,12525536,3.0,2,1270.0,6037,33759601.0,-118115692.0,978900.0,6.037578e+07,46298,3101,96237,1974,172592.0,426404.0,2016,253812.0,5140.38,6.037578e+13,0.010327,2017-04-25,Los Angeles


### Stat Tests

- pearsonr: lot size and log error
- ttest: LA and Ventura and log_error
- pearsonr: tax_amount and log_erro

###### Lot size and log_error (pearsonr)

- HO:Lot Size and log_error has not linear relation.
- H1:Lot Size and log_error has linear relation.

In [17]:
α = 0.05
corr, p = stats.pearsonr(train.lot_size_square_feet, train.log_error)
corr, p

if p < α:
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")
print(p)

We reject the null hypothesis
0.002887407387085765


###### LA and Ventura and log_error (ttest)

- H0: log_error for Los Angeles equals log_error for Ventura.

- Ha: log_error for Los Angeles doesnot equal log_error for Ventura.

In [26]:
ventura_sample = train[train.county == 'Ventura'].log_error

In [27]:
los_angeles_sample = train[train.county == 'Los Angeles'].log_error

In [28]:
t, p = stats.ttest_ind(ventura_sample, los_angeles_sample, equal_var=False)

t, p

if p < α:
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")
print(p)


We fail to reject the null hypothesis
0.9907438540390221


###### tax_amount and log_error(pearsonr)

- HO:tax_amount and log_error has not linear relation.
- H1:tax_amount and log_error has linear relation.

In [18]:
α = 0.05
corr, p = stats.pearsonr(train.tax_amount, train.log_error)
corr, p

if p < α:
    print("We reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")
print(p)

We fail to reject the null hypothesis
0.9221442669347352
